In [1]:
import pandas as pd
from langdetect import detect
import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [2]:
def convert_unidecode(x):
    return unidecode.unidecode(x)

In [3]:
# remove ponctuation
def remove_punct1(x):
    p = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', "'", '"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']
    return ' '.join([string for string in x.split(' ') if string not in p])

In [4]:
# remove ponctuation
def remove_punct2(x):
    import re
    return re.sub(r'[^\w]|[^\w] ', ' ', x)

In [5]:
from googletrans import Translator
translator = Translator()

In [6]:
def translation(x):
    if detect(x) != 'en':
        x = translator.translate(x).text
        return x
    else:
        return x

In [7]:
# Define stop words to clean text (job description)
stopwords_en = set(stopwords.words('english'))

stopwords_pt = set(stopwords.words('portuguese'))

stopwords_es = set(stopwords.words('spanish'))

stopwords_de = set(stopwords.words('german'))

stopwords_nl = set(stopwords.words('dutch'))

stopwords_fr = set(stopwords.words('french'))

stopwords_fi = set(stopwords.words('finnish'))

In [8]:
def remove_stop_words(x):
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_en)]) if detect(x)=='en'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_pt)]) if detect(x)=='pt'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_es)]) if detect(x)=='es'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_de)]) if detect(x)=='de'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fr)]) if detect(x)=='fr'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_nl)]) if detect(x)=='nl'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fi)])

In [9]:
def remove_stop_words1(x):
    stop_words_concat = list(stopwords_en)+list(stopwords_es)+list(stopwords_pt)
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in stop_words_concat])

In [10]:
def stem_snowball(x):
    if detect(x) == 'pt':
        stemmer = SnowballStemmer('portuguese')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'en':
        stemmer = SnowballStemmer('english')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'es':
        stemmer = SnowballStemmer('spanish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'de':
        stemmer = SnowballStemmer('german')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'nl':
        stemmer = SnowballStemmer('dutch')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fr':
        stemmer = SnowballStemmer('french')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fi':
        stemmer = SnowballStemmer('finnish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    else:
        return nltk.word_tokenize(x)

In [11]:
def bigrams(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.bigrams(x))
        return output
    except:
        pass

In [12]:
def trigram(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.trigrams(x))
        return output
    except:
        pass

In [13]:
def token(x):
    return nltk.word_tokenize(x)

In [14]:
df = pd.read_csv('/home/inrx/Ironhack/TA/job-search/linkedin-project/linkedin_search.csv')
print (df.shape)

(794, 12)


In [15]:
df = df.drop_duplicates(keep='last')
df = df.reset_index(drop=True)
df.shape

(592, 12)

In [16]:
df['lang'] = df['role-description'].apply(lambda x: detect(x))

df = df[df['lang'].map(df['lang'].value_counts()) > 2]

In [17]:
df = df.replace(to_replace=r'\n', value=' ', regex=True)

In [18]:
df['role-description'] = df['role-description'].apply(remove_punct1)
df['role-description'] = df['role-description'].apply(remove_punct2)

In [19]:
df['role-description'] = df['role-description'].apply(lambda x: x.lower())

In [20]:
df['role_description1'] = df['role-description'].apply(translation)

In [21]:
df.role_description1

0      looking for a date analyst for analysis and pa...
1      as we embark on an exciting period of change t...
2      are you a bit mad  very driven  and excited ab...
3      for our office loft in berlin friedrichshain w...
4        the company   a leading provider of research...
5      overview  the firm is looking to recruit a dat...
6      job title  data analyst location  bradford sal...
7      our data team in bratislava is looking for tal...
8      for the last 50 years  our customers have refe...
9      we re looking for people with  master degree i...
10     our company  at adobe  you will be immersed in...
11     overview  the business of fashion is seeking a...
12     business analyst  global data transformation  ...
13     dataiku s mission is big  to enable all people...
14     mycs means creating furniture you love exactly...
15     first iwantic Spanish agency specializing in s...
16     we are  the wix business analysis  ba  group  ...
17     do you love data and its

In [22]:
df['role_description1'] = df['role_description1'].apply(convert_unidecode)

In [23]:
# Clean the text according remove the stopwords according to the language of the job post
df['role_description1'] = df['role_description1'].apply(remove_stop_words)

# Clean the text according remove the stopwords regardless the language of the job post
df['role_description1'] = df['role_description1'].apply(remove_stop_words1)

In [24]:
df_stem = df[['link', 'role_description1']].copy()

In [25]:
df_stem['role_description1'] = df_stem['role_description1'].apply(token)

In [26]:
df_stem['role_des_bigrams'] = df_stem['role_description1'].apply(bigrams)

In [27]:
df_stem['role_des_trigrams'] = df_stem['role_description1'].apply(trigram)

In [28]:
df_stem.head()

,link,role_description1,role_des_bigrams,role_des_trigrams
0,https://www.linkedin.com/jobs/view/1605083546,"[looking, date, analyst, analysis, pattern, ex...","[(looking, date), (date, analyst), (analyst, a...","[(looking, date, analyst), (date, analyst, ana..."
1,https://www.linkedin.com/jobs/view/1564171840,"[embark, exciting, period, change, throughout,...","[(embark, exciting), (exciting, period), (peri...","[(embark, exciting, period), (exciting, period..."
2,https://www.linkedin.com/jobs/view/1599766365,"[bit, mad, driven, excited, building, new, thi...","[(bit, mad), (mad, driven), (driven, excited),...","[(bit, mad, driven), (mad, driven, excited), (..."
3,https://www.linkedin.com/jobs/view/1599163575,"[office, loft, berlin, friedrichshain, looking...","[(office, loft), (loft, berlin), (berlin, frie...","[(office, loft, berlin), (loft, berlin, friedr..."
4,https://www.linkedin.com/jobs/view/1603033659,"[company, leading, provider, research, data, s...","[(company, leading), (leading, provider), (pro...","[(company, leading, provider), (leading, provi..."


In [114]:
%store df_stem

Stored 'df_stem' (DataFrame)


In [29]:
trigrams_list = [trigram for lst in df_stem.role_des_trigrams for trigram in lst]

In [85]:
trigrams_list = [tuple(sorted (trigram)) for trigram in trigrams_list]

In [86]:
trigrams = {tup:trigrams_list.count(tup) for tup in trigrams_list}

#trigrams = {k: v for k, v in trigrams.items() if v > 2}

trigrams_sorted = sorted(trigrams.items(), key=lambda kv: kv[1], reverse=True)

In [117]:
%store trigrams_list
%store trigrams 
%store trigrams_sorted

Stored 'trigrams_list' (list)
Stored 'trigrams' (dict)
Stored 'trigrams_sorted' (list)


In [47]:
bigrams_list = [bigram for lst in df_stem.role_des_bigrams for bigram in lst]

In [49]:
bigrams = {tup:bigrams_list.count(tup) for tup in bigrams_list}


{('looking', 'date'): 1,
 ('date', 'analyst'): 3,
 ('analyst', 'analysis'): 1,
 ('analysis', 'pattern'): 1,
 ('pattern', 'extraction'): 1,
 ('extraction', 'milano'): 1,
 ('milano', 'seat'): 1,
 ('seat', 'candidates'): 1,
 ('candidates', 'required'): 1,
 ('required', 'basic'): 1,
 ('basic', 'knowledge'): 20,
 ('knowledge', 'standard'): 4,
 ('standard', 'languages'): 1,
 ('languages', 'sql'): 14,
 ('sql', 'python'): 55,
 ('python', 'also'): 3,
 ('also', 'welcome'): 1,
 ('welcome', 'knowledge'): 1,
 ('knowledge', 'following'): 7,
 ('following', 'technologists'): 1,
 ('technologists', 'p'): 1,
 ('p', 'consulting'): 1,
 ('embark', 'exciting'): 3,
 ('exciting', 'period'): 3,
 ('period', 'change'): 1,
 ('change', 'throughout'): 1,
 ('throughout', 'business'): 4,
 ('business', 'need'): 5,
 ('need', 'new'): 1,
 ('new', 'data'): 31,
 ('data', 'analyst'): 539,
 ('analyst', 'join'): 66,
 ('join', 'thriving'): 1,
 ('thriving', 'inventory'): 1,
 ('inventory', 'function'): 1,
 ('function', 'excellent

In [50]:
#bigrams = {k: v for k, v in bigrams.items() if v > 2}

bigrams_sorted = sorted(bigrams.items(), key=lambda kv: kv[1], reverse=True)

In [118]:
%store bigrams_list
%store bigrams
%store bigrams_sorted

Stored 'bigrams_list' (list)
Stored 'bigrams' (dict)
Stored 'bigrams_sorted' (list)


In [32]:
tokens_list = [word for lst in df_stem.role_description1 for word in lst]

tokens = {word:tokens_list.count(word) for word in tokens_list}

tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

In [33]:
tokens = {word:tokens_list.count(word) for word in tokens_list}

In [34]:
tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

In [119]:
%store tokens_list
%store tokens
%store tokens_sorted

Stored 'tokens_list' (list)
Stored 'tokens' (dict)
Stored 'tokens_sorted' (list)


In [51]:
bigrams_sorted

[(('data', 'analyst'), 539),
 (('data', 'analysis'), 337),
 (('data', 'analytics'), 332),
 (('data', 'science'), 229),
 (('business', 'intelligence'), 192),
 (('big', 'data'), 192),
 (('data', 'driven'), 186),
 (('communication', 'skills'), 168),
 (('machine', 'learning'), 159),
 (('problem', 'solving'), 158),
 (('experience', 'data'), 153),
 (('computer', 'science'), 148),
 (('experience', 'working'), 136),
 (('data', 'sets'), 125),
 (('data', 'visualization'), 125),
 (('years', 'experience'), 115),
 (('power', 'bi'), 107),
 (('data', 'quality'), 102),
 (('decision', 'making'), 97),
 (('analytical', 'skills'), 95),
 (('skills', 'ability'), 90),
 (('python', 'r'), 88),
 (('ad', 'hoc'), 86),
 (('ability', 'work'), 86),
 (('skills', 'experience'), 84),
 (('data', 'management'), 84),
 (('attention', 'detail'), 83),
 (('strong', 'analytical'), 80),
 (('data', 'sources'), 78),
 (('internal', 'external'), 77),
 (('2', 'years'), 77),
 (('analytics', 'team'), 75),
 (('r', 'python'), 72),
 (('f

In [101]:
trigrams_sorted

[(('analyst', 'data', 'join'), 64),
 (('problem', 'skills', 'solving'), 53),
 (('analyst', 'data', 'looking'), 48),
 (('analysis', 'data', 'experience'), 47),
 (('bi', 'power', 'tableau'), 47),
 (('analyst', 'business', 'data'), 46),
 (('python', 'r', 'sql'), 45),
 (('business', 'data', 'intelligence'), 45),
 (('employer', 'equal', 'opportunity'), 45),
 (('data', 'tools', 'visualization'), 43),
 (('computer', 'engineering', 'science'), 41),
 (('analytical', 'problem', 'solving'), 40),
 (('english', 'spoken', 'written'), 40),
 (('analytical', 'skills', 'strong'), 37),
 (('analyst', 'data', 'experience'), 37),
 (('2', 'experience', 'years'), 35),
 (('communication', 'good', 'skills'), 34),
 (('communication', 'verbal', 'written'), 33),
 (('analyst', 'data', 'role'), 33),
 (('communication', 'excellent', 'skills'), 33),
 (('communication', 'skills', 'written'), 32),
 (('data', 'experience', 'working'), 31),
 (('3', 'experience', 'years'), 31),
 (('computer', 'mathematics', 'science'), 31)

In [108]:
tokens_sorted

[('data', 5562),
 ('business', 1933),
 ('experience', 1695),
 ('team', 1513),
 ('work', 1221),
 ('skills', 1077),
 ('analytics', 959),
 ('analysis', 937),
 ('working', 841),
 ('knowledge', 804),
 ('analyst', 770),
 ('management', 677),
 ('role', 638),
 ('tools', 624),
 ('sql', 608),
 ('strong', 597),
 ('new', 587),
 ('development', 573),
 ('ability', 540),
 ('support', 540),
 ('company', 526),
 ('analytical', 516),
 ('reporting', 463),
 ('insights', 457),
 ('teams', 456),
 ('solutions', 442),
 ('us', 424),
 ('science', 420),
 ('product', 419),
 ('environment', 411),
 ('develop', 409),
 ('help', 408),
 ('information', 407),
 ('technical', 405),
 ('looking', 402),
 ('using', 397),
 ('python', 394),
 ('key', 389),
 ('clients', 387),
 ('make', 377),
 ('within', 374),
 ('customer', 374),
 ('projects', 372),
 ('opportunity', 369),
 ('job', 361),
 ('understanding', 358),
 ('quality', 357),
 ('world', 349),
 ('bi', 347),
 ('provide', 347),
 ('based', 344),
 ('across', 342),
 ('people', 341),
 

In [38]:
functions = [word.strip() for word in df['function'] if word !='']

In [39]:
functions_dict = {word:functions.count(word) for word in functions}
functions_dict = sorted(functions_dict.items(), key=lambda kv: kv[1], reverse=True)

In [40]:
import re
def separate_capitalized(lst):
    s_list = re.findall('[A-Z][^A-Z]*', lst)
    ss_list = [nlst.split(' ') for nlst in s_list]
    return (' '.join([word.title() for lst in ss_list for word in lst if word != '']))

In [41]:
separate_capitalized(functions_dict)

TypeError: expected string or bytes-like object

In [ ]:
functions_dict

In [ ]:
df['function'][0]